# **03_DataExploration**

## Objectives

* To conduct exploratory data analysis (EDA) to understand which factors are associated with XX to meet business requirement one (Client is interested in determining which demographic, lifestyle, medical and symptom factors are associated with clinical measures of disease, in terms of FEV1 and FVC)

## Inputs

* Raw data generated from "01_DataCollection". Saved as: inputs\datasets\raw\asthma_disease_data.csv

## Outputs

* Generate code that answers business requirement one, and can be used to build the Streamlit App

---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'/workspaces/AsthmaBurden/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'/workspaces/AsthmaBurden'

# Load Data

Load the data and drop "DoctorInCharge" as not needed for analyses.

In [7]:
import pandas as pd
import numpy as np

df_raw_path = "outputs/datasets/cleaned/asthma_disease_data_cleaned.csv"
df = pd.read_csv(df_raw_path)
df.head(10)


,PatientID,Age,Gender,Ethnicity,EducationLevel,BMI,Smoking,PhysicalActivity,DietQuality,SleepQuality,...,GastroesophagealReflux,LungFunctionFEV1,LungFunctionFVC,Wheezing,ShortnessOfBreath,ChestTightness,Coughing,NighttimeSymptoms,ExerciseInduced,Diagnosis
0,5034,63.0,0,1,0,15.848744,0,0.894448,5.488696,8.701003,...,0,1.369051,4.941206,0,0,1,0,0,1,0
1,5035,26.0,1,2,2,22.757042,0,5.897329,6.341014,5.153966,...,0,2.197767,1.702393,1,0,0,1,1,1,0
2,5036,57.0,0,2,1,18.395396,0,6.739367,9.196237,6.840647,...,0,1.698011,5.022553,1,1,1,0,1,1,0
3,5037,40.0,1,2,1,38.515278,0,1.404503,5.826532,4.253036,...,0,3.032037,2.300159,1,0,1,1,1,0,0
4,5038,61.0,0,0,3,19.283802,0,4.604493,3.127048,9.625799,...,0,3.470589,3.067944,1,1,1,0,0,1,0
5,5039,21.0,0,2,0,21.812975,0,0.470044,1.759118,9.549262,...,0,2.328191,5.898515,1,0,1,0,0,1,0
6,5040,45.0,1,1,1,30.245954,1,9.371784,7.030507,5.746128,...,0,2.995100,1.701512,1,1,1,0,0,0,0
7,5041,26.0,0,0,1,26.048416,1,8.344096,1.626484,6.431179,...,0,2.069343,4.012260,1,0,0,1,1,1,0
8,5042,49.0,1,1,2,32.676204,0,2.690256,3.920034,5.843645,...,1,1.761242,5.190931,1,1,1,1,0,0,0
9,5043,45.0,1,1,1,29.910298,0,2.895720,2.607700,7.234908,...,0,2.848420,5.771022,1,0,0,0,1,1,0


---

# Data Exploration

Section 2 content

---

NOTE

* You may add as many sections as you want, as long as it supports your project workflow.
* All notebook's cells should be run top-down (you can't create a dynamic wherein a given point you need to go back to a previous cell to execute some task, like go back to a previous cell and refresh a variable content)

---

# Push files to Repo

* In case you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
  # create here your folder
  # os.makedirs(name='')
except Exception as e:
  print(e)
